In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

### Read and clean beauty data 
Data exported from experimental software, we clean and format it for analysis here.


Key columns to focus on are:
- **Participant Public ID**, as you can link that to the demographic data on the other file
- **Response**, which is the beauty rating (1 to 10)
- **Absolute Reaction Time** – this is in ms. So should always be at least 10,000 (as the videos were 10 s long). If for any subject it is consistently smaller than this, then the subjects effectively cheated and you should consider removing that subject’s data.
- **Emergence** – this is the emergence value for that video. Typically it will be negative. But if it’s less negative or a little positive, that is more emergent.

Things to watch out for:

- Reaction times too short – see above
- Subject didn’t complete the experiment – each subject has 316 video files to see. The way the output goes, each subject will have two lines per trial, so each subject SHOULD have over 630 lines of the spreadsheet. If it is significantly shorter than this, you should consider removing that subject from your analysis.

In [2]:
df = pd.read_csv(
    '../subj-data/aesthetics/data_exp_125666-v4_task-2z6n.csv',
    skipfooter = 1)

/home/m/.local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
# remove columns that are the same for the whole df
remove = [] 
for c in df.columns:
    vals = df[c].dropna().unique()
    if len(vals) <= 1:
        remove.append(c)
df = df.drop(remove, axis = 1)

In [4]:
# remove columns about irrelevant data
df = df.drop(['Schedule ID', 'Participant Private ID',
       'Participant External Session ID', 'Participant Device Type',
       'Participant Device', 'Participant OS', 'Participant Browser',
       'Participant Monitor Size', 'Participant Viewport Size',
       'Display', 'Screen ID', 'Object Number', 'Object ID',
       'Component Name', ], axis = 1)
# remove timestamp columns
df = df.drop([  
    'UTC Timestamp', 'UTC Date', 'Local Timestamp',
    'Local Timezone', 'Local Date' ], axis = 1)
# remove reaction time columns, keep only absolute
df = df.drop([
    'Clock Time', 'Onset Time', 'Reaction Time', 'Response Onset',
    'Absolute Onset Time', 'Absolute Clock Time'
    ], axis = 1)

In [5]:
df

,Event Index,Participant Public ID,Participant Status,Trial Number,Response Type,Response,Absolute Reaction Time,Object Name,Spreadsheet: display,Spreadsheet: File_List,Spreadsheet: Emergence
0,1,5ffe0751a3b25844a41e8d1d,complete,1,continue,NaN,7418.975,Continue Button,Instructions,NaN,NaN
1,2,5ffe0751a3b25844a41e8d1d,complete,1,continue,NaN,16530.957,Continue Button,MainVids2,B_8-3.mp4,-6.786704
2,3,5ffe0751a3b25844a41e8d1d,complete,1,tooEarly,6.0,16533.757,BeautySlider,MainVids2,B_8-3.mp4,-6.786704
3,4,5ffe0751a3b25844a41e8d1d,complete,2,continue,NaN,60.865,Continue Button,MainVids2,A_8-26.mp4,-1.270255
4,5,5ffe0751a3b25844a41e8d1d,complete,2,tooEarly,NaN,61.165,BeautySlider,MainVids2,A_8-26.mp4,-1.270255
...,...,...,...,...,...,...,...,...,...,...,...
46071,630,602a2911a73ba046955f6c58,complete,315,continue,NaN,14860.603,Continue Button,MainVids2,B_2-5.mp4,-3.735530
46072,631,602a2911a73ba046955f6c58,complete,315,tooEarly,3.0,14864.703,BeautySlider,MainVids2,B_2-5.mp4,-3.735530
46073,632,602a2911a73ba046955f6c58,complete,316,continue,NaN,12965.065,Continue Button,MainVids2,B_2-6.mp4,-4.846682
46074,633,602a2911a73ba046955f6c58,complete,316,tooEarly,4.0,12965.065,BeautySlider,MainVids2,B_2-6.mp4,-4.846682


### Remove subjects with incomplete data
We expect each subject to have 316 responses and participant status of `complete`. We thus remove the subjects who have less than 315 trial numbers as seen below, we can see those are the same ones not labelled as `complete`.

In [6]:
trials = pd.DataFrame(
    df.groupby('Participant Public ID')['Trial Number'].nunique())
trials[trials['Trial Number'] < 316]

,Trial Number
Participant Public ID,
55a809fffdf99b752061ad7c,144
5964fca32fa2630001d88601,315
5cd338927d0b180016b32626,257
5e8341865cf99101d3a4a9ff,55
60101181d3a08f26a98a23e2,7
604055c26be513259eced8fd,288
632750037efc7dd0fe0facaa,146
63bd9367a2b5cb5ed0957afb,315
63d5aa69fbbf387649d6c99a,301


In [7]:
incomplete = pd.DataFrame(df[df['Participant Status'] != 'complete']\
    .groupby('Participant Public ID')['Trial Number'].nunique())
incomplete

,Trial Number
Participant Public ID,
55a809fffdf99b752061ad7c,144
5cd338927d0b180016b32626,257
5e8341865cf99101d3a4a9ff,55
60101181d3a08f26a98a23e2,7
604055c26be513259eced8fd,288
632750037efc7dd0fe0facaa,146
63d5aa69fbbf387649d6c99a,301
640be287f0b00e45af92d6e3,285


In [8]:
beau_df = df.copy()
beau_df = beau_df[beau_df['Participant Status'] == 'complete']
beau_df = beau_df.drop(['Participant Status'], axis = 1)

In [9]:
beau_df

,Event Index,Participant Public ID,Trial Number,Response Type,Response,Absolute Reaction Time,Object Name,Spreadsheet: display,Spreadsheet: File_List,Spreadsheet: Emergence
0,1,5ffe0751a3b25844a41e8d1d,1,continue,NaN,7418.975,Continue Button,Instructions,NaN,NaN
1,2,5ffe0751a3b25844a41e8d1d,1,continue,NaN,16530.957,Continue Button,MainVids2,B_8-3.mp4,-6.786704
2,3,5ffe0751a3b25844a41e8d1d,1,tooEarly,6.0,16533.757,BeautySlider,MainVids2,B_8-3.mp4,-6.786704
3,4,5ffe0751a3b25844a41e8d1d,2,continue,NaN,60.865,Continue Button,MainVids2,A_8-26.mp4,-1.270255
4,5,5ffe0751a3b25844a41e8d1d,2,tooEarly,NaN,61.165,BeautySlider,MainVids2,A_8-26.mp4,-1.270255
...,...,...,...,...,...,...,...,...,...,...
46071,630,602a2911a73ba046955f6c58,315,continue,NaN,14860.603,Continue Button,MainVids2,B_2-5.mp4,-3.735530
46072,631,602a2911a73ba046955f6c58,315,tooEarly,3.0,14864.703,BeautySlider,MainVids2,B_2-5.mp4,-3.735530
46073,632,602a2911a73ba046955f6c58,316,continue,NaN,12965.065,Continue Button,MainVids2,B_2-6.mp4,-4.846682
46074,633,602a2911a73ba046955f6c58,316,tooEarly,4.0,12965.065,BeautySlider,MainVids2,B_2-6.mp4,-4.846682


### Remove irrelevant responses
We are only interested in the beauty ratings, so we also remove all entries where 'Object Name' is not `BeautySlider`, which also have a `NaN` response. This should leave a single value in the 'Spreadsheet: Display' column which should be `MainVids2`. 

We observe also that the 'Response Type' column only contains `tooEarly` so we remove it as well.

In [10]:
beau_df = beau_df[beau_df['Object Name'] == 'BeautySlider']
beau_df = beau_df.drop(['Object Name'], axis = 1)

if len(beau_df['Spreadsheet: display'].unique()) == 1:
    beau_df = beau_df.drop(['Spreadsheet: display'], axis = 1)

if len(beau_df['Response Type'].unique()) == 1:
    beau_df = beau_df.drop(['Response Type'], axis = 1)

In [11]:
beau_df

,Event Index,Participant Public ID,Trial Number,Response,Absolute Reaction Time,Spreadsheet: File_List,Spreadsheet: Emergence
2,3,5ffe0751a3b25844a41e8d1d,1,6.0,16533.757,B_8-3.mp4,-6.786704
4,5,5ffe0751a3b25844a41e8d1d,2,NaN,61.165,A_8-26.mp4,-1.270255
6,7,5ffe0751a3b25844a41e8d1d,3,4.0,14221.802,A_2-13.mp4,-8.711708
8,9,5ffe0751a3b25844a41e8d1d,4,NaN,66.354,B_1-11.mp4,-3.364134
10,11,5ffe0751a3b25844a41e8d1d,5,2.0,15704.275,B_6-10.mp4,-9.822086
...,...,...,...,...,...,...,...
46066,625,602a2911a73ba046955f6c58,312,7.0,11890.280,A_10-7.mp4,-0.050254
46068,627,602a2911a73ba046955f6c58,313,5.0,10988.780,A_4-1.mp4,-2.435160
46070,629,602a2911a73ba046955f6c58,314,6.0,14748.654,B_5-16.mp4,-14.658225
46072,631,602a2911a73ba046955f6c58,315,3.0,14864.703,B_2-5.mp4,-3.735530


### Remove nulls and responses with too small reaction times
We test the absolute reaction time is less than 10s (10000ms), and whether this matches with the 'Response' being `NaN`. If so, all those entries are removed. 

We see that there are ~9000 responses below 10s, of which ~7200 are not null. There are 2500 nulls of which 843 are in responses > 10000. Before dropping them we count responses and nulls per subject. 

In [12]:
len(beau_df['Participant Public ID'].unique())

68

In [13]:
print(f"All responses:   {len(beau_df)}")
print(f"Responses < 10s: {len(beau_df[beau_df['Absolute Reaction Time'] < 10000])}")
print(f"Null responses:  {len(beau_df[beau_df['Response'].isna()])}")
print(f"Null < 10s:      {len(beau_df[beau_df['Absolute Reaction Time'] < 10000][beau_df['Response'].isna()])}")
print(f"Null > 10s:      {len(beau_df[beau_df['Absolute Reaction Time'] >= 10000][beau_df['Response'].isna()])}")

All responses:   21486
Responses < 10s: 8896
Null responses:  2512
Null < 10s:      1669
Null > 10s:      843


In [14]:
counts = pd.DataFrame(
    beau_df.groupby('Participant Public ID')['Response'].count())
counts['Response < 10'] = beau_df[beau_df['Absolute Reaction Time'] < 10000]\
    .groupby(beau_df['Participant Public ID'])['Response'].count()
counts['Null'] = beau_df.Response.isna().groupby(
    beau_df['Participant Public ID']).sum()
counts

,Response,Response < 10,Null
Participant Public ID,,,
55da0d9369dbc30005b684c3,295,23.0,21
581f6c63930a280001062e8c,297,2.0,19
58aca85e0da7f10001de92d4,212,168.0,104
5964fca32fa2630001d88601,226,3.0,88
59d3958ed552520001fe6dfc,252,NaN,64
...,...,...,...
63e5dd2b9eeb6c9f6a4e82d2,249,2.0,67
63ea5007fd3a4fed03b929d1,302,86.0,14
63ed455be60b8094c15477e8,282,34.0,34


To investigate if any subjects need to be removed from the set entirely, we will check if more than 20% of their responses are either null or under the response threshold.

In [15]:
print(len(counts[counts['Response'] / 5 <= counts['Response < 10']]))

36


In [16]:
counts[counts['Response'] / 5 <= counts['Response < 10']]

,Response,Response < 10,Null
Participant Public ID,,,
58aca85e0da7f10001de92d4,212,168.0,104
5b4b679dff74970001256d36,310,222.0,6
5b6449eff7798c000182782a,141,115.0,175
5bead56b5324b10001c2551c,315,91.0,1
5c29ca6e867f660001aed60f,315,227.0,1
5c3d48955fd1050001a99364,213,95.0,103
5d32005bc117ec0001d1f83c,293,207.0,23
5e70bd5480f43a0009625d4c,309,290.0,7
5e93641d8028056ac36b16d9,315,288.0,1


In [17]:
beau_nona = beau_df.dropna(subset = ['Response'], 
                           axis = 0, inplace = False)
beau_nona = beau_nona[beau_df['Absolute Reaction Time'] > 9000]

Having filtered the rows we now cleanup the dataframe to obtain group name and segment index from the video filename.

In [18]:
beau_nona['Group'] = beau_nona['Spreadsheet: File_List'].apply(
    lambda r: r.split('-')[0])
beau_nona['Segment'] = beau_nona['Spreadsheet: File_List'].apply(
    lambda r: r.split('-')[1].split('.')[0])
beau_nona = beau_nona.drop(['Event Index', 'Trial Number', 'Absolute Reaction Time',
                            'Spreadsheet: File_List'], axis = 1)

In [19]:
beau_means = pd.DataFrame(beau_nona.groupby(
    ['Group', 'Segment'])['Response'].mean())
beau_means['mean_beauty'] = beau_means.Response
beau_means['std_beauty'] = beau_nona.groupby(
    ['Group', 'Segment'])['Response'].std()
beau_means = beau_means.drop('Response', axis = 1)

In [20]:
beau_means

mean_beauty  std_beauty
Group Segment                         
A_10  1           5.413793    2.457020
      2           6.315789    1.960492
      3           5.800000    2.244081
      4           5.894737    2.413806
      5           5.414634    2.729245
...                    ...         ...
B_9   5           4.476190    2.431883
      6           5.595238    2.420033
      7           5.461538    2.246004
      8           6.023256    2.424953
      9           3.333333    1.756620

[316 rows x 2 columns]

### Merge with the rest of segment data

In [33]:
df_segment = pd.read_csv('GERF_segment_data.csv')
df_segment.Segment = df_segment.Segment.astype(str)
df_segment = df_segment.set_index(['Group', 'Segment'])

In [34]:
df_segment = df_segment.join(beau_means, how='outer')
df_segment

mean_psi   std_psi  mean_dist_cmass  err_mean_dist_cmass  \
Group Segment                                                             
A_10  1             NaN       NaN         0.314680             0.018500   
      2             NaN       NaN         0.321147             0.018207   
      3             NaN       NaN         0.306074             0.021114   
      4             NaN       NaN         0.313716             0.035409   
      5             NaN       NaN         0.299343             0.024350   
...                 ...       ...              ...                  ...   
B_9   5       -5.437223  1.043389         0.257162             0.025933   
      6       -4.096789  1.802614         0.242599             0.018825   
      7       -5.374901  1.369893         0.253116             0.012448   
      8       -7.794946  0.752205         0.239227             0.020298   
      9       -3.670128  0.807396         0.281714             0.030317   

               var_dist_cmass  err_var_dist_cmass  mean_dist_nearest  \
Group Segment                                                          
A_10  1              0.007542            0.003297           0.180106   
      2              0.004464            0.003003           0.161891   
      3              0.006096            0.003297           0.150428   
      4              0.005761            0.003306           0.160672   
      5              0.010973            0.003393           0.160013   
...                       ...                 ...                ...   
B_9   5              0.012502            0.001725           0.111721   
      6              0.013597            0.001919           0.093411   
      7              0.012787            0.002501           0.102723   
      8              0.015225            0.002634           0.086288   
      9              0.015868            0.003534           0.136448   

               err_mean_dist_nearest  var_angle  err_var_angle  vicsek_order  \
Group Segment                                                                  
A_10  1                     0.013511   5.855703       3.261516      0.226226   
      2                     0.023536   5.909399       3.830762      0.189950   
      3                     0.019530   4.954254       3.601364      0.264359   
      4                     0.013023   6.411343       4.181316      0.224526   
      5                     0.015735   4.575335       2.572113      0.204265   
...                              ...        ...            ...           ...   
B_9   5                     0.023238   8.354182       6.512136      0.670361   
      6                     0.007244   2.041463       3.249479      0.670558   
      7                     0.011023   8.482427       6.147049      0.665505   
      8                     0.008782   2.929246       4.253619      0.688679   
      9                     0.018911   6.698286       4.496148      0.454060   

               err_vicsek_order  mean_beauty  std_beauty  
Group Segment                                             
A_10  1                0.088179     5.413793    2.457020  
      2                0.097909     6.315789    1.960492  
      3                0.098714     5.800000    2.244081  
      4                0.107664     5.894737    2.413806  
      5                0.118158     5.414634    2.729245  
...                         ...          ...         ...  
B_9   5                0.309995     4.476190    2.431883  
      6                0.313307     5.595238    2.420033  
      7                0.276459     5.461538    2.246004  
      8                0.317673     6.023256    2.424953  
      9                0.237001     3.333333    1.756620  

[316 rows x 14 columns]

In [35]:
df_segment.to_csv('GERF_beauty_segment_data.csv')